In [50]:
import boto3
import json

region_aws = 'us-east-1'

ec2 = boto3.resource('ec2', region_name=region_aws)
ec2_client = boto3.client('ec2', region_name=region_aws)
iam = boto3.client('iam')
ssm = boto3.client('ssm')

In [2]:
# Create a policy
policy_name = 'Cookbook-106-SSM-Role'
policy_document = {
  "Version": "2012-10-17",
  "Statement": [
  {
    "Effect": "Allow",
    "Principal": {
      "Service": "ec2.amazonaws.com"
    },
    "Action": "sts:AssumeRole"
  }
  ]
}

ssm_role = iam.create_role(
    RoleName=policy_name,
    AssumeRolePolicyDocument=json.dumps(policy_document)
)

In [3]:
ssm_role_arn = ssm_role['Role']['Arn']

In [4]:
# Attach the policy to the role
response = iam.attach_role_policy(
    RoleName=policy_name,
    PolicyArn='arn:aws:iam::aws:policy/AmazonSSMManagedInstanceCore'
)

{'ResponseMetadata': {'RequestId': '6019a3c3-b10c-4834-a268-f6b3b3510897',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '6019a3c3-b10c-4834-a268-f6b3b3510897',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Thu, 06 Oct 2022 15:59:00 GMT'},
  'RetryAttempts': 0}}

In [54]:
instance_profile_name = 'Cookbook-109-SSM-Role'

# Create an instance profile
instance_profile = iam.create_instance_profile(
    InstanceProfileName=instance_profile_name
)
# Add the role that you created to the instance profile:
response = iam.add_role_to_instance_profile(
    InstanceProfileName=instance_profile_name,
    RoleName=policy_name
)

In [57]:
# Select a vpc
vpc_id = 'vpc-0fc47b87065ca097a'
vpc = ec2.Vpc(vpc_id)

# Select a subnet
subnet = list(vpc.subnets.all())[0]

In [44]:
for sn in vpc.subnets.all():
    #print(sn.id + " - " + sn.availability_zone)
    subnet = sn

In [58]:
# Create security group enable HHTPS:
security_group_https = vpc.create_security_group(
    GroupName='AWSCookbook-SG-HTTPS',
    Description='Allow HTTPS access',
    VpcId=vpc.id,
)

ec2_client.authorize_security_group_ingress(
    GroupId=security_group_https.id,
    IpPermissions=[
        {'FromPort': 443,
            'IpProtocol': 'tcp',
            'IpRanges': [
                {
                    'CidrIp': vpc.cidr_block,
                    'Description': 'Allow HTTPS access'
                }
            ],
            'ToPort': 443
        }
    ]
)

{'Return': True,
 'SecurityGroupRules': [{'SecurityGroupRuleId': 'sgr-053bb7e6910bc2877',
   'GroupId': 'sg-0c4718dd53013bbc2',
   'GroupOwnerId': '255224756231',
   'IsEgress': False,
   'IpProtocol': 'tcp',
   'FromPort': 443,
   'ToPort': 443,
   'CidrIpv4': '10.1.0.0/16',
   'Description': 'Allow HTTPS access'}],
 'ResponseMetadata': {'RequestId': 'f8b6ccc8-efa2-4b3d-a94f-e5eb7ac108fd',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f8b6ccc8-efa2-4b3d-a94f-e5eb7ac108fd',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '781',
   'date': 'Thu, 06 Oct 2022 16:48:15 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [59]:
# Enable Dns in VPC
vpc.modify_attribute(EnableDnsSupport={'Value': True})
vpc.modify_attribute(EnableDnsHostnames={'Value': True})

{'ResponseMetadata': {'RequestId': 'e1a2d511-62db-42e9-9953-87900f444220',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e1a2d511-62db-42e9-9953-87900f444220',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '237',
   'date': 'Thu, 06 Oct 2022 16:48:26 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [60]:
# Creates a VPC endpoint for SSM:
vpc_endpoint_ssm = ec2_client.create_vpc_endpoint(
    VpcId=vpc.id,
    ServiceName='com.amazonaws.' + region_aws + '.ssm',
    VpcEndpointType='Interface',
    SubnetIds=[subnet.id],
    PrivateDnsEnabled=True,
    SecurityGroupIds=[security_group_https.id]
)
# Creates a VPC endpoint for EC2 messages:
vpc_endpoint_ec2msg = ec2_client.create_vpc_endpoint(
    VpcId=vpc.id,
    ServiceName='com.amazonaws.' + region_aws + '.ec2messages',
    VpcEndpointType='Interface',
    SubnetIds=[subnet.id],
    PrivateDnsEnabled=True,
    SecurityGroupIds=[security_group_https.id]
)
# Creates a VPC endpoint for SSMMessages:
vpc_endpoint_ssm = ec2_client.create_vpc_endpoint(
    VpcId=vpc.id,
    ServiceName='com.amazonaws.' + region_aws + '.ssmmessages',
    VpcEndpointType='Interface',
    SubnetIds=[subnet.id],
    PrivateDnsEnabled=True,
    SecurityGroupIds=[security_group_https.id]
)

In [61]:
# Consulte en SSM el último ID de AMI de Amazon Linux 2 disponible en su región y guárdelo como variable de entorno:
ssm_response = ssm.get_parameter(
    Name='/aws/service/ami-amazon-linux-latest/amzn2-ami-hvm-x86_64-gp2'
)

In [63]:
# Create a key pair
key_name = 'AWSCookbook-KeyPair-Instance'
if key_name not in [key_pair.key_name for key_pair in ec2.key_pairs.all()]:
    key_pair = ec2.create_key_pair(KeyName=key_name)
    key_pair_file = open(key_name + '.pem', 'w')
    key_pair_file.write(key_pair.key_material)
    key_pair_file.close()

In [65]:
instances = ec2.create_instances(
    ImageId=ssm_response['Parameter']['Value'],
    MinCount=1,
    MaxCount=1,
    InstanceType='t2.micro',
    KeyName=key_name,
    SubnetId=subnet.id,
    IamInstanceProfile={
        'Name': instance_profile_name
    },
    TagSpecifications=[
        {
            'ResourceType': 'instance',
            'Tags': [
                {
                    'Key': 'Name',
                    'Value': 'Cookbook-SSM-Instance'
                },
            ]
        },
    ]
)

In [ ]:
# Creates a VPC endpoint for SSM:
vpc_endpoint_ssm = ec2_client.create_vpc_endpoint(
    VpcId=vpc.id,
    ServiceName='com.amazonaws.' + region_aws + '.ssm',
    VpcEndpointType='Interface',
    SubnetIds=[subnet.id],
    PrivateDnsEnabled=True,
    SecurityGroupIds=[security_group_https.id]
)

In [ ]:
# Creates a VPC endpoint for EC2 messages:
vpc_endpoint_ec2msg = ec2_client.create_vpc_endpoint(
    VpcId=vpc.id,
    ServiceName='com.amazonaws.' + region_aws + '.ec2messages',
    VpcEndpointType='Interface',
    SubnetIds=[subnet.id],
    PrivateDnsEnabled=True,
    SecurityGroupIds=[security_group_https.id]
)

In [ ]:
# Creates a VPC endpoint for SSMMessages:
vpc_endpoint_ssm = ec2_client.create_vpc_endpoint(
    VpcId=vpc.id,
    ServiceName='com.amazonaws.' + region_aws + '.ssmmessages',
    VpcEndpointType='Interface',
    SubnetIds=[subnet.id],
    PrivateDnsEnabled=True,
    SecurityGroupIds=[security_group_https.id]
)

In [56]:
instance_id = ssm.describe_instance_information()['InstanceInformationList'][0]['InstanceId']
print("En la terminal ejecute el siguiente comando:")
print("   aws ssm start-session --target " + instance_id)

En la terminal ejecute el siguiente comando:
   aws ssm start-session --target i-01b1df1b6208cf192
